# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121539e+02     1.619388e+00
 * time: 0.44199085235595703
     1     1.157098e+01     9.253537e-01
 * time: 2.519850015640259
     2    -1.219857e+01     9.739805e-01
 * time: 3.261586904525757
     3    -3.392480e+01     7.418835e-01
 * time: 4.317023038864136
     4    -4.756827e+01     5.244027e-01
 * time: 5.394103050231934
     5    -5.690073e+01     2.308065e-01
 * time: 6.464671850204468
     6    -5.969967e+01     1.649586e-01
 * time: 7.203001022338867
     7    -6.081694e+01     5.976932e-02
 * time: 7.951946973800659
     8    -6.128722e+01     5.009918e-02
 * time: 8.665868997573853
     9    -6.157411e+01     5.510973e-02
 * time: 9.39097285270691
    10    -6.180673e+01     3.360346e-02
 * time: 10.106185913085938
    11    -6.197774e+01     2.648181e-02
 * time: 10.819478988647461
    12    -6.206219e+01     1.668673e-02
 * time: 11.532532930374146
    13    -6.211602e+01     1.331856e-02
 * time: 12.245954036712646
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671064
    AtomicLocal         -18.8557558
    AtomicNonlocal      14.8522547
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485346 
    Xc                  -19.3336812

    total               -62.261666453237
